In [1]:
import cv2
import numpy as np
import keras
from LinearClassifier import LinearClassifier
import os

Using TensorFlow backend.


# Loading Images

In [2]:
armyManagement = cv2.imread('./MasterOpenCV/FoE/Army/ArmyManagement.png')
armyManagement_cut = cv2.imread('./MasterOpenCV/FoE/Army/ArmyManagement_cut.png')
armyManagement_hurt = cv2.imread('./MasterOpenCV/FoE/Army/ArmyManagement_HurtUnits.png')

hover_segment = cv2.imread('./MasterOpenCV/FoE/Army/FoundUnits/patch_00.jpg')
rogue_segment = cv2.imread('./MasterOpenCV/FoE/Army/FoundUnits/patch_10.jpg')

hoverTank_1 = cv2.imread('./MasterOpenCV/FoE/Army/HoverTank.png')
hoverTank_2 = cv2.imread('./MasterOpenCV/FoE/Army/HoverTank_2.png')
Rogue = cv2.imread('./MasterOpenCV/FoE/Army/Rogue.png')

LC = LinearClassifier(Rogue= Rogue, Hover= hoverTank_1)
classifier = LC.train_classifier()

# Class for army management

In [50]:
class units_management():
    def __init__(self, unitID, unitLives, bounds):
        self.unitID = unitID
        self.unitLives = unitLives
        self.needsReplacement = self.unitLives < 5
        self.unitBounds = bounds
    def __str__(self):
        return f'{self.unitID} with {self.unitLives} lives and {self.needsReplacement} with {self.unitBounds}'

In [51]:
class army_management():
    def __init__(self, armyManagement, model):
        self.armyManagement = armyManagement
        self.model = model
        self.units = []
        
        self.alive_units = 0
        
        self.processArmy()
        
    def lives_mask(self, frame):
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) 
        lowerBound = np.array([35, 90, 150])
        upperBound = np.array([45, 210, 240])
        mask = cv2.inRange(hsv, lowerBound, upperBound) 
        result = cv2.bitwise_and(frame, frame, mask = mask)
        result = cv2.pyrUp(result)
        result = cv2.pyrUp(result)
        return result

    def lives_count(self, image):
        im = self.lives_mask(image)
        edged = cv2.Canny(im, 50, 250)
        _, contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        return len(contours)
    
    def processArmy(self):
        # img = self.armyManagement[40:-10, 10:280]
        img = self.armyManagement
        sizeX, sizeY = img.shape[1], img.shape[0]

        nRows = 2
        mCols = 4
        
        for j in range(0, mCols):
            for i in range(0,nRows):
                y_higher = i*int(sizeY/nRows)
                y_lower = i*int(sizeY/nRows) + int(sizeY/nRows)
                x_left = j*int(sizeX/mCols)
                x_right = j*int(sizeX/mCols) + int(sizeX/mCols)
                roi = img[y_higher: y_lower,
                          x_left:x_right]
                
                lives_c = self.lives_count(roi)
                if lives_c:
                    unitID = self.model.predict(self.preprocess_data(roi))
                    bounds_ = ((y_higher, x_left), (y_lower, x_right))
                    
                    tempUnit = units_management(unitID= unitID,
                                                unitLives=lives_c,
                                                bounds=bounds_)
                    self.units.append(tempUnit)
                # cv2.imwrite('./MasterOpenCV/FoE/Army/FoundUnits/patch_'+str(i)+str(j)+".jpg", roi)
            
    def preprocess_data(self, image):
        image_toresize = image.copy()
        height, width = image_toresize.shape[:2]
        blank_image = np.zeros((80, 80, 3), np.uint8)
        blank_image[:,:] = (255,255,255)
        l_img = blank_image.copy()
        x_offset = 40 - int(width/2)
        y_offset = 40 - int(height/2)
        l_img[y_offset:y_offset+height, x_offset:x_offset+width] = image_toresize.copy()
        return np.array(l_img[np.newaxis, :, :, :])

In [52]:
armyManagement_cut = cv2.imread('./MasterOpenCV/FoE/Army/ArmyManagement_cut.png')
army = army_management(armyManagement=armyManagement_cut, model=classifier)

In [53]:
for _ in army.units:
    print(_)

[[1.]] with 3 lives and True with ((0, 0), (71, 66))
[[1.]] with 1 lives and True with ((71, 0), (142, 66))
[[0.]] with 10 lives and False with ((0, 66), (71, 132))
[[0.]] with 10 lives and False with ((71, 66), (142, 132))
[[0.]] with 10 lives and False with ((0, 132), (71, 198))
[[0.]] with 10 lives and False with ((71, 132), (142, 198))
[[0.]] with 10 lives and False with ((0, 198), (71, 264))
[[0.]] with 10 lives and False with ((71, 198), (142, 264))


## Old Project

In [ ]:
mainCity = cv2.imread('./MasterOpenCV/FoE/Something.png')
aiding = cv2.imread('./MasterOpenCV/FoE/Aid.png')
FT = cv2.imread('./MasterOpenCV/FoE/FT.png')
GB = cv2.imread('./MasterOpenCV/FoE/GB.png')
MPed = cv2.imread('./MasterOpenCV/FoE/Motivated.png')
barrack = cv2.imread('./MasterOpenCV/FoE/Barrack.png')

(toBeAided, noAid) = template_finder(mainCity, aiding, count=True)
print(noAid)
friendTavern = template_finder(mainCity, FT)
GB = template_finder(mainCity, GB)
Motivated = template_finder(mainCity, MPed, threshold=0.5)
Troops = template_finder(mainCity, barrack)

cv2.imshow('Ready troops', Troops)
cv2.waitKey()
cv2.imshow('Aiding', toBeAided)
cv2.waitKey()
cv2.imshow('Friend Tavern', friendTavern)
cv2.waitKey()
cv2.imshow('Great Buildings', GB)
cv2.waitKey()
cv2.imshow('Motivated', Motivated)
cv2.waitKey()

cv2.destroyAllWindows()